In [1]:
import tensorflow as tf
from tensorflow.keras import layers
from tensorflow import keras
import pandas as pd
import numpy as np
from sklearn.feature_extraction.text import TfidfVectorizer
import collections
from sklearn.feature_extraction import text
import requests
import joblib
import os

In [2]:
def download_file_from_google_drive(id, destination):
    URL = "https://docs.google.com/uc?export=download"

    session = requests.Session()

    response = session.get(URL, params = { 'id' : id }, stream = True)
    token = get_confirm_token(response)

    if token:
        params = { 'id' : id, 'confirm' : token }
        response = session.get(URL, params = params, stream = True)

    save_response_content(response, destination)    

def get_confirm_token(response):
    for key, value in response.cookies.items():
        if key.startswith('download_warning'):
            return value

    return None

def save_response_content(response, destination):
    CHUNK_SIZE = 32768

    with open(destination, "wb") as f:
        for chunk in response.iter_content(CHUNK_SIZE):
            if chunk: # filter out keep-alive new chunks
                f.write(chunk)

In [18]:
def predict_sentiment(model_name="model_sa.h5", labels="labels.pkl", vectorizer="vectorizer.pkl"):
    
    model_url = "https://drive.google.com/file/d/1c-GOhXDZXF5NJDYoAwDpc6EJSm_zY4UM/view?usp=sharing"
    file_id = "1c-GOhXDZXF5NJDYoAwDpc6EJSm_zY4UM"
    destination = "model_sa.h5"
    labels = joblib.load(labels)
    vectorizer = joblib.load(vectorizer)
    df = pd.read_csv("Evaluation-dataset.csv", header=None)
    result = []
    
    if not os.path.exists(model_name):
        download_file_from_google_drive(file_id, destination)
        print("Downloading model from: ", model_url)
    else:
        print("Model Found.")
    
    model = keras.models.load_model(model_name)
    
    for sent_i, sent in enumerate(df[0][0:10]):
        x_new_vec = vectorizer.transform([sent])
        x_new = x_new_vec.toarray()
        y_proba = model.predict(x_new)
        for idx, y_prob in enumerate(y_proba[0]):
            if round(y_prob, 2) > 0.9:
                result.append(labels[idx])
        print(str(sent_i+1) + ": SENTENCE: ", sent, "\nPredicted Labels: ", result, "\nActual Labels: ", [df[1][sent_i], df[2][sent_i], df[3][sent_i]], "\n\n")
        result = []    

In [19]:
predict_sentiment()

Model Found.
1: SENTENCE:  Tires where delivered to the garage of my choice,the garage notified me when they had been delivered. A day and time was arranged with the garage and I went and had them fitted,a Hassel free experience. 
Predicted Labels:  [] 
Actual Labels:  ['garage service positive', 'ease of booking positive', nan] 


2: SENTENCE:  Easy Tyre Selection Process, Competitive Pricing and Excellent Fitting Service 
Predicted Labels:  ['value for money positive'] 
Actual Labels:  ['garage service positive', 'value for money positive', nan] 


3: SENTENCE:  Very easy to use and good value for money. 
Predicted Labels:  ['value for money positive'] 
Actual Labels:  ['value for money positive', nan, nan] 


4: SENTENCE:  Really easy and convenient to arrange 
Predicted Labels:  ['ease of booking positive'] 
Actual Labels:  ['ease of booking positive', nan, nan] 


5: SENTENCE:  It was so easy to select tyre sizes and arrange local fitting. The prices were competitive 
Predicted La